# Customer Churn Prediction 
Machine Learning Project - Decison Tree Classification 

Authors:
* José Marcos Leal Barbosa Filho
* Lucas Ismael Campos Medeiros

Institution: Universidade Federal do Rio Grande do Norte - Brazil.

## Dataset Description

It is a **customer churn** modeling dataset containing 10.000 rows (each representing an unique customer) with 14 columns: 13 general features and one target feature (**Exited**). The data is composed of both numerical and categorical features:

**Numeric Features:**

    RowNumber: The sequence number of the rows. 
    CustomerId: A unique ID of the customer.
    CreditScore: The credit score of the customer,
    Age: The age of the customer,
    Tenure: The number of months the client has been with the firm.
    Balance: Balance remaining in the customer account,
    NumOfProducts: The number of products purchased by the customer.
    EstimatedSalary: The estimated salary of the customer.

**Categorical Features:**

    Surname: The surname of the customer.
    Geography: The country of the customer.
    Gender: M/F
    HasCrCard: Whether the customer has a credit card or not.
    IsActiveMember: Whether the customer is active or not.

**The target column:** 

    Exited — Whether the customer churned or not.

The dataset can be seen and downloaded [here](https://www.kaggle.com/datasets/aakash50897/churn-modellingcsv?resource=download).

## Load Libraries

In [ ]:
!pip install wandb
!pip install pytest pytest-sugar
!pip install pandas-profiling==3.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 181 kB 64.1 MB/s 
     |████████████████████████████████| 145 kB 66.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=07acbd7f423335b5b51e8c4d882e2a7f35d68e22fd19e69825dbd0621441bb81
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pytest-sugar: filename=pytest_sugar-0.9.4-py2.py3-none-any.whl size=8985 sha256=0f1f3a564bbb6761027394fa6982862edb57c2873a7244c568d94d15fe19ef92
  Stored in directory: /root/.cache/pip/wheels/9c/e8/b6/5009ec756a2f40eed690a8b0a95549cd788c1ec968832876df
Successful

In [ ]:
import wandb
import logging
import tempfile
import os
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

-----------------------------------

## Login to wandb

In [ ]:
!wandb login --relogin

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1.Extract, Transform and Load (ETL)

### 1.1.Fetch Data

In [ ]:
# columns used
columns = ['RowNumber', 'CustomerId', 'Surname', 'CreditScore',
           'Geography', 'Gender', 'Age', 'Tenure', 
           'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
           'EstimatedSalary', 'Exited']
# importing the dataset
churndf = pd.read_csv("https://github.com/x4nth055/pythoncode-tutorials/raw/master/machine-learning/customer-churn-detection/Churn_Modelling.csv",
                      header=None,
                      names=columns)
churndf.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,1,15634602,Hargrave,619,France,Female,42,2,0,1,1,1,101348.88,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,3,15619304,Onio,502,France,Female,42,8,159660.8,3,1,0,113931.57,1
4,4,15701354,Boni,699,France,Female,39,1,0,2,0,0,93826.63,0


* It was chosen to remove the following columns:

  * **RowNumber:** Indicates only the sequence number of the lines;
  * **CustomerId:** High cardinality column with 10,000 unique IDs;
  * **Surname:** Column with high cardinality, showing the last names of each consumer.

In [ ]:
# removing unecessary columns and reseting indexes
churndf = churndf.drop([0,])
churndf.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)
churndf.reset_index(drop=True,inplace=True)
churndf.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.8,3,1,0,113931.57,1
3,699,France,Female,39,1,0,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0


In [ ]:
# saving to csv
churndf.to_csv("raw_data.csv", index=False)

In [ ]:
# Saving artifact to wandb
!wandb artifact put \
       --name churn_prediction_project/raw_data.csv \
       --type raw_data \
       --description "Customer Churn" raw_data.csv

wandb: Uploading file raw_data.csv to: "eec1509/churn_prediction_project/raw_data.csv:latest" (raw_data)
wandb: Currently logged in as: macleal (eec1509). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.17
wandb: Run data is saved locally in /content/wandb/run-20220529_141637-upueb7xb
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run polar-vortex-1
wandb: ⭐️ View project at https://wandb.ai/eec1509/churn_prediction_project
wandb: 🚀 View run at https://wandb.ai/eec1509/churn_prediction_project/runs/upueb7xb
Artifact uploaded, use this artifact in a run by adding:

    artifact = run.use_artifact("eec1509/churn_prediction_project/raw_data.csv:latest")

wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: Synced polar-vortex-1: https://wandb.ai/eec1509/churn_prediction_project/runs/upueb7xb
wandb: Synced 5 W&B file(s), 0 media file(s), 1

----------------

### 1.2.Exploratory Data Analysis (EDA)

In [ ]:
# save_code tracking all changes of the notebook and sync with Wandb
run = wandb.init(project="churn_prediction_project", save_code=True)

wandb: Currently logged in as: macleal (eec1509). Use `wandb login --relogin` to force relogin


In [ ]:
# download the latest version of artifact raw_data.csv
artifact = run.use_artifact("churn_prediction_project/raw_data.csv:latest")

# create a dataframe from the artifact
df = pd.read_csv(artifact.file())

#### 1.2.1 Pandas Profiling

In [ ]:
ProfileReport(df, title= "Pandas Profiling Report", explorative=True)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
run.finish()

--------------------------

### 1.3.Preprocessing

In [ ]:
input_artifact="churn_prediction_project/raw_data.csv:latest"
artifact_name="preprocessed_data.csv"
artifact_type="clean_data"
artifact_description="Data after preprocessing"

In [ ]:
# create a new job_type
run = wandb.init(project="churn_prediction_project", job_type="process_data")

In [ ]:
# download the latest version of artifact raw_data.csv
artifact=run.use_artifact(input_artifact)

# create a dataframe from the artifact
df = pd.read_csv(artifact.file())

In [ ]:
# delete duplicated rows
df.drop_duplicates(inplace=True)

# generate a "clean data file"
df.to_csv(artifact_name, index=False)

In [ ]:
# Create a new artifact and configure with the necessary arguments
artifact = wandb.Artifact(name=artifact_name,
                         type=artifact_type,
                         description=artifact_description)
artifact.add_file(artifact_name)

<ManifestEntry digest: 2VNPzyBON65Yp9cxPORlnA==>

In [ ]:
run.log_artifact(artifact)

In [ ]:
run.finish()

----------------------------

## 2.Data Check

In [ ]:
%%file test_data.py
import pytest
import wandb
import pandas as pd

# This is global so all tests are collected under the same run
run = wandb.init(project="churn_prediction_project", job_type="data_checks")

@pytest.fixture(scope="session")
def data():

    local_path = run.use_artifact("churn_prediction_project/preprocessed_data.csv:latest").file()
    df = pd.read_csv(local_path)

    return df

def test_data_length(data):
    """
    We test that we have enough data to continue
    """
    assert len(data) > 1000


def test_number_of_columns(data):
    """
    We test that we have enough data to continue
    """
    assert data.shape[1] == 11

def test_column_presence_and_type(data):

    required_columns = {
        #"CustomerId": pd.api.types.is_int64_dtype,
        #"Surname": pd.api.types.is_object_dtype,
        "CreditScore": pd.api.types.is_int64_dtype,
        "Geography": pd.api.types.is_object_dtype,
        "Gender": pd.api.types.is_object_dtype,
        "Age": pd.api.types.is_int64_dtype,
        "Tenure": pd.api.types.is_int64_dtype,
        "Balance": pd.api.types.is_float_dtype,
        "NumOfProducts": pd.api.types.is_int64_dtype,
        "HasCrCard": pd.api.types.is_int64_dtype,
        "IsActiveMember": pd.api.types.is_int64_dtype,
        "EstimatedSalary": pd.api.types.is_float_dtype,  
        "Exited": pd.api.types.is_int64_dtype
    }

    # Check column presence
    assert set(data.columns.values).issuperset(set(required_columns.keys()))

    for col_name, format_verification_funct in required_columns.items():

        assert format_verification_funct(data[col_name]), f"Column {col_name} failed test {format_verification_funct}"


def test_class_names(data):

    # Check that only the known classes are present
    known_classes = [
        0,
        1
    ]

    assert data["Exited"].isin(known_classes).all()


def test_column_ranges(data):

    ranges = {
        "CreditScore": (0, 1000),
        "Age": (0,100),
        "Tenure": (0,10),
        "Balance": (0, 1.484705e+06),
        "NumOfProducts": (1,4),
        "HasCrCard": (0,1),
        "IsActiveMember": (0,1),
        "EstimatedSalary": (0, 1.484705e+06),
        "Exited": (0, 1)
    }

    for col_name, (minimum, maximum) in ranges.items():

        assert data[col_name].dropna().between(minimum, maximum).all(), (
            f"Column {col_name} failed the test. Should be between {minimum} and {maximum}, "
            f"instead min={data[col_name].min()} and max={data[col_name].max()}"
        )

Writing test_data.py


In [ ]:
!pytest . -vv

Test session starts (platform: linux, Python 3.7.13, pytest 3.6.4, pytest-sugar 0.9.4)
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1, sugar-0.9.4

 test_data.py::test_data_length ✓                                 20% ██        
 test_data.py::test_number_of_columns ✓                           40% ████      
 test_data.py::test_column_presence_and_type ✓                    60% ██████    
 test_data.py::test_class_names ✓                                 80% ████████  
 test_data.py::test_column_ranges ✓                              100% ██████████

Results (5.67s):
       5 passed


In [ ]:
run.finish()

----------------------------------------

## 3.Data Segregation

In [ ]:
# global variables

# ratio - 70% train / 30% test
test_size = 0.30

# seed used to reproduce purposes
seed = 42

# reference (column) to stratify the data
stratify = "Exited"

# name of the input artifact
artifact_input_name = "churn_prediction_project/preprocessed_data.csv:latest"

# type of the artifact
artifact_type = "segregated_data"

In [ ]:
# configure logging 
logging.basicConfig(level=logging.INFO,
                   format="%(asctime)s %(message)s",
                   datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging object
logger = logging.getLogger()

# init wandb project
run = wandb.init(project="churn_prediction_project", job_type="split_data")

logger.info("Downloading and reading artifact")
artifact=run.use_artifact(artifact_input_name)
artifact_path=artifact.file()
df = pd.read_csv(artifact_path)

# Split in train/test
logger.info("Splitting data into train and test")
splits = {}

splits["train"], splits["test"] = train_test_split(df,
                                                  test_size=test_size,
                                                  random_state=seed,
                                                  stratify=df[stratify])

29-05-2022 14:18:27 Downloading and reading artifact
29-05-2022 14:18:28 Splitting data into train and test


In [ ]:
# Save artifacts
with tempfile.TemporaryDirectory() as tmp_dir:
    
    for split, df in splits.items():
        
        # Make the artifact name from the name of the split plus the provided root
        artifact_name = f"{split}.csv"
        
        # Get the path on disk within the temp directory
        temp_path = os.path.join(tmp_dir,artifact_name)
        
        logger.info(f"Uploading the {split} dataset to {artifact_name}")
        
        # Save then upload to W&B
        df.to_csv(temp_path,index=False)
        
        artifact = wandb.Artifact(name=artifact_name,
                                 type=artifact_type,
                                 description=f"{split} split of dataset {artifact_input_name}")
        artifact.add_file(temp_path)
        
        logger.info("Logging artifact")
        run.log_artifact(artifact)
        
        artifact.wait()

29-05-2022 14:18:28 Uploading the train dataset to train.csv
29-05-2022 14:18:28 Logging artifact
29-05-2022 14:18:29 Uploading the test dataset to test.csv
29-05-2022 14:18:29 Logging artifact


In [ ]:
run.finish()

------------------------

## 4.Trainning

### 4.1.Holdout Configuration

In [ ]:
# global variables

# ratio used to split train and validation data
val_size = 0.30

# seed used to reproduce purposes
seed = 42

# reference (column) to stratify the data
stratify = "Exited"

# name of the input artifact
artifact_input_name = "churn_prediction_project/train.csv:latest"

# type of the artifact
artifact_type = "Train"

In [ ]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

# initiate the wandb project
run = wandb.init(project="churn_prediction_project",job_type="train")

logger.info("Downloading and reading train artifact")
local_path = run.use_artifact(artifact_input_name).file()
df_train = pd.read_csv(local_path)

# Spliting train.csv into train and validation dataset
logger.info("Spliting data into train/val")
# split-out train/validation and test dataset
x_train, x_val, y_train, y_val = train_test_split(df_train.drop(labels=stratify,axis=1),
                                                  df_train[stratify],
                                                  test_size=val_size,
                                                  random_state=seed,
                                                  shuffle=True,
                                                  stratify=df_train[stratify])

29-05-2022 14:18:42 Downloading and reading train artifact
29-05-2022 14:18:43 Spliting data into train/val


In [ ]:
logger.info("x train: {}".format(x_train.shape))
logger.info("y train: {}".format(y_train.shape))
logger.info("x val: {}".format(x_val.shape))
logger.info("y val: {}".format(y_val.shape))

29-05-2022 14:18:43 x train: (4900, 10)
29-05-2022 14:18:43 y train: (4900,)
29-05-2022 14:18:43 x val: (2100, 10)
29-05-2022 14:18:43 y val: (2100,)


### 4.2.Data Preparation


#### 4.2.1.Outlier Removal

In [ ]:
logger.info("Outlier Removal")
# temporary variable
x = x_train.select_dtypes("int64").copy()

# identify outlier in the dataset
lof = LocalOutlierFactor()
outlier = lof.fit_predict(x)
mask = outlier != -1

29-05-2022 14:18:43 Outlier Removal


In [ ]:
logger.info("x_train shape [original]: {}".format(x_train.shape))
logger.info("x_train shape [outlier removal]: {}".format(x_train.loc[mask,:].shape))

29-05-2022 14:18:43 x_train shape [original]: (4900, 10)
29-05-2022 14:18:43 x_train shape [outlier removal]: (4877, 10)


In [ ]:
logger.info("y_train shape [original]: {}".format(y_train.shape))
logger.info("y_train shape [outlier removal]: {}".format(y_train.loc[mask].shape))

29-05-2022 14:18:43 y_train shape [original]: (4900,)
29-05-2022 14:18:43 y_train shape [outlier removal]: (4877,)


In [ ]:
# AVOID data leakage and you should not do this procedure in the preprocessing stage
# Note that we did not perform this procedure in the validation set
x_train = x_train.loc[mask,:].copy()
y_train = y_train[mask].copy()

#### 4.2.2.Target Variable Encoding

In this case, the target variable is already encoded, but let's create an encoder to transform the numeric variable into categorical.

In [ ]:
logger.info("Encoding a Target Variable")

# define a categorical encoding for target variable
le = LabelEncoder()
le.fit(["Contiuned", "Exited"])
teste = le.inverse_transform(y_train)

29-05-2022 14:18:43 Encoding a Target Variable


-----------------------

### 4.3.Creating a Full-Pipeline

#### 4.3.1.Transformers

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    # Class Constructor
    def __init__(self, feature_names):
        self.feature_names = feature_names
        
    # Return self nothing to do here
    def fit(self, X, y=None):
        return self
    
    # Method that describes what tis custom transformer need to do
    def transform(self, X, y=None):
        return X[self.feature_names]
    
# Handling categorical features
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes one boolean as argument
    def __init__(self, new_features=True, colnames=None):
        self.new_features = new_features
        self.colnames = colnames
        
    # Return self nothing else to do here
    def fit(self, X, y=None):
        return self
    
    def get_feature_names_out(self):
        return self.colnames.tolist()
    
    # Transformer method we wrote for this transformer
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # Remove white space in categorical features
        df = df.apply(lambda row: row.str.strip())
        
        # customize feature?
        # How can I identify what needs to be modified? EDA!!!!!!
        if self.new_features:
            
            # replace ? with unknown
            edit_cols = ['Geography', 'Gender']
            for col in edit_cols:
                df.loc[df[col].str.contains("\?"), col] = 'unknown'
        
        # update column names
        self.colnames = df.columns
        df = pd.DataFrame(X, columns=self.colnames)
        
        return df

# transform numerical features
class NumericalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes a model parameter as its argument
    # model 0: minmax
    # model 1: standard
    # model 2: without scaler
    def __init__(self, model=0, colnames=None):
        self.model = model
        self.colnames = colnames
        self.scaler = None

    # Fit is used only to learn statistical about Scalers
    def fit(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # minmax
        if self.model == 0:
            self.scaler = MinMaxScaler()
            self.scaler.fit(df)
        # standard scaler
        elif self.model == 1:
            self.scaler = StandardScaler()
            self.scaler.fit(df)
        return self

    # return columns names after transformation
    def get_feature_names_out(self):
        return self.colnames

    # Transformer method we wrote for this transformer
    # Use fitted scalers
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # update columns name
        self.colnames = df.columns.tolist()

        # minmax
        if self.model == 0:
            # transform data
            df = self.scaler.transform(df)
        elif self.model == 1:
            # transform data
            df = self.scaler.transform(df)
        else:
            df = df.values

        return df

#### 4.3.2.Holdout Training Pipeline

In [ ]:
# model = 0 (min-max), 1 (z-score), 2 (without normalization)
numerical_model = 2

# Categrical features to pass down the categorical pipeline
categorical_features = x_train.select_dtypes("object").columns.to_list()

# Numerical features to pass down the numerical pipeline
numerical_features = x_train.select_dtypes(["int64","float"]).columns.to_list()

# Defining the steps for the categorical pipeline
categorical_pipeline = Pipeline(steps=[('cat_selector', FeatureSelector(categorical_features)),
                                       ('imputer_cat', SimpleImputer(strategy="most_frequent")),
                                       ('cat_transformer', CategoricalTransformer(colnames=categorical_features)),
                                       # ('cat_encoder','passthrough'
                                       ('cat_encoder', OneHotEncoder(sparse=False, drop="first"))])

# Defining the steps in the numerical pipeline
numerical_pipeline = Pipeline(steps=[('num_selector', FeatureSelector(numerical_features)),
                                     ('imputer_num', SimpleImputer(strategy="median")),
                                     ('num_transformer', NumericalTransformer(numerical_model, 
                                                                              colnames=numerical_features))])

# Combine numerical and categorical pieplines into one full big pipeline horizontally
full_pipeline_preprocessing = FeatureUnion(transformer_list=[('cat_pipeline', categorical_pipeline),
                                                             ('num_pipeline', numerical_pipeline)])

#### 4.3.3. Trainning

In [ ]:
# The full pipeline 
pipe = Pipeline(steps = [('full_pipeline', full_pipeline_preprocessing),
                         ("classifier",DecisionTreeClassifier())])

# training
logger.info("Training")
pipe.fit(x_train, y_train)

# predict
logger.info("Infering")
predict = pipe.predict(x_val)

# Evaluation Metrics
logger.info("Evaluation metrics")
fbeta = fbeta_score(y_val, predict, beta=1, zero_division=1)
precision = precision_score(y_val, predict, zero_division=1)
recall = recall_score(y_val, predict, zero_division=1)
acc = accuracy_score(y_val, predict)

logger.info("Accuracy: {}".format(acc))
logger.info("Precision: {}".format(precision))
logger.info("Recall: {}".format(recall))
logger.info("F1: {}".format(fbeta))

29-05-2022 14:18:43 Training
29-05-2022 14:18:43 Infering
29-05-2022 14:18:43 Evaluation metrics
29-05-2022 14:18:43 Accuracy: 0.7957142857142857
29-05-2022 14:18:43 Precision: 0.4989293361884368
29-05-2022 14:18:43 Recall: 0.544392523364486
29-05-2022 14:18:43 F1: 0.5206703910614525


In [ ]:
run.finish()

--------------------------

### 4.4.Hyperparameter Tuning

In [ ]:
seed = 42

In [ ]:
sweep_config = {
    # try grid or random
    "method": "grid",
    "metric": {
        "name": "Accuracy",
        "goal": "maximize"
        },
    "parameters": {
        "criterion": {
            "values": ["gini","entropy"]
            },
        "splitter": {
            "values": ["random","best"]
        },
        "model": {
            "values": [0,1,2]
        },
        "random_state": {
            "values": [seed]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="churn_prediction_project")

Create sweep with ID: ael79lao
Sweep URL: https://wandb.ai/eec1509/churn_prediction_project/sweeps/ael79lao


In [ ]:
def train():
    with wandb.init() as run:

        # The full pipeline 
        pipe = Pipeline(steps = [('full_pipeline', full_pipeline_preprocessing),
                                    ("classifier",DecisionTreeClassifier())
                                    ]
                        )

        # update the parameters of the pipeline that we would like to tuning
        pipe.set_params(**{"full_pipeline__num_pipeline__num_transformer__model": run.config.model})
        pipe.set_params(**{"classifier__criterion": run.config.criterion})
        pipe.set_params(**{"classifier__splitter": run.config.splitter})
        pipe.set_params(**{"classifier__random_state": run.config.random_state})

        # training
        logger.info("Training")
        pipe.fit(x_train, y_train)

        # predict
        logger.info("Infering")
        predict = pipe.predict(x_val)

        # Evaluation Metrics
        logger.info("Evaluation metrics")
        fbeta = fbeta_score(y_val, predict, beta=1, zero_division=1)
        precision = precision_score(y_val, predict, zero_division=1)
        recall = recall_score(y_val, predict, zero_division=1)
        acc = accuracy_score(y_val, predict)

        logger.info("Accuracy: {}".format(acc))
        logger.info("Precision: {}".format(precision))
        logger.info("Recall: {}".format(recall))
        logger.info("F1: {}".format(fbeta))

        run.summary["Accuracy"] = acc
        run.summary["Precision"] = precision
        run.summary["Recall"] = recall
        run.summary["F1"] = fbeta

In [ ]:
wandb.agent(sweep_id, train, count=12)

wandb: Agent Starting Run: h20xy71m with config:
wandb: 	criterion: gini
wandb: 	model: 0
wandb: 	random_state: 42
wandb: 	splitter: random


29-05-2022 14:19:00 Training
29-05-2022 14:19:00 Infering
29-05-2022 14:19:00 Evaluation metrics
29-05-2022 14:19:00 Accuracy: 0.7785714285714286
29-05-2022 14:19:00 Precision: 0.45951859956236324
29-05-2022 14:19:01 Recall: 0.49065420560747663
29-05-2022 14:19:01 F1: 0.47457627118644063


Accuracy,0.77857
F1,0.47458
Precision,0.45952
Recall,0.49065


wandb: Agent Starting Run: w92vlgev with config:
wandb: 	criterion: gini
wandb: 	model: 0
wandb: 	random_state: 42
wandb: 	splitter: best


29-05-2022 14:19:21 Training
29-05-2022 14:19:21 Infering
29-05-2022 14:19:21 Evaluation metrics
29-05-2022 14:19:21 Accuracy: 0.7947619047619048
29-05-2022 14:19:21 Precision: 0.49682875264270615
29-05-2022 14:19:21 Recall: 0.5490654205607477
29-05-2022 14:19:21 F1: 0.5216426193118757


...

29-05-2022 14:22:25 Training
29-05-2022 14:22:25 Infering
29-05-2022 14:22:25 Evaluation metrics
29-05-2022 14:22:25 Accuracy: 0.7871428571428571
29-05-2022 14:22:25 Precision: 0.4789356984478936
29-05-2022 14:22:25 Recall: 0.5046728971962616
29-05-2022 14:22:25 F1: 0.49146757679180886


Accuracy,0.78714
F1,0.49147
Precision,0.47894
Recall,0.50467


wandb: Agent Starting Run: 193ucjlu with config:
wandb: 	criterion: entropy
wandb: 	model: 2
wandb: 	random_state: 42
wandb: 	splitter: best


29-05-2022 14:22:46 Training
29-05-2022 14:22:46 Infering
29-05-2022 14:22:46 Evaluation metrics
29-05-2022 14:22:46 Accuracy: 0.8
29-05-2022 14:22:46 Precision: 0.5088495575221239
29-05-2022 14:22:46 Recall: 0.5373831775700935
29-05-2022 14:22:46 F1: 0.5227272727272727


Accuracy,0.8
F1,0.52273
Precision,0.50885
Recall,0.53738


--------------------------

### 4.5.Configure and Train the best model

In [ ]:
# initiate the wandb project
run = wandb.init(project="churn_prediction_project",job_type="train")

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


In [ ]:
# The full pipeline 
pipe = Pipeline(steps = [('full_pipeline', full_pipeline_preprocessing),
                         ("classifier",DecisionTreeClassifier())
                         ]
                )

# update the parameters of the pipeline that we would like to tuning
pipe.set_params(**{"full_pipeline__num_pipeline__num_transformer__model": 1})
pipe.set_params(**{"classifier__criterion": 'entropy'})
pipe.set_params(**{"classifier__splitter": 'best'})
pipe.set_params(**{"classifier__random_state": 42})



# training
logger.info("Training")
pipe.fit(x_train, y_train)

# predict
logger.info("Infering")
predict = pipe.predict(x_val)

# Evaluation Metrics
logger.info("Evaluation metrics")
fbeta = fbeta_score(y_val, predict, beta=1, zero_division=1)
precision = precision_score(y_val, predict, zero_division=1)
recall = recall_score(y_val, predict, zero_division=1)
acc = accuracy_score(y_val, predict)

logger.info("Accuracy: {}".format(acc))
logger.info("Precision: {}".format(precision))
logger.info("Recall: {}".format(recall))
logger.info("F1: {}".format(fbeta))

run.summary["Acc"] = acc
run.summary["Precision"] = precision
run.summary["Recall"] = recall
run.summary["F1"] = fbeta

29-05-2022 14:22:58 Training
29-05-2022 14:22:58 Infering
29-05-2022 14:22:58 Evaluation metrics
29-05-2022 14:22:58 Accuracy: 0.8004761904761905
29-05-2022 14:22:58 Precision: 0.5099778270509978
29-05-2022 14:22:58 Recall: 0.5373831775700935
29-05-2022 14:22:58 F1: 0.5233219567690558


In [ ]:
# Get categorical column names
cat_names = pipe.named_steps['full_pipeline'].get_params()["cat_pipeline"][3].get_feature_names_out().tolist()
cat_names

['Geography_Germany', 'Geography_Spain', 'Gender_Male']

In [ ]:
# Get numerical column names
num_names = pipe.named_steps['full_pipeline'].get_params()["num_pipeline"][2].get_feature_names_out()
num_names

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [ ]:
# merge all column names together
all_names = cat_names + num_names
all_names

['Geography_Germany',
 'Geography_Spain',
 'Gender_Male',
 'CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [ ]:
# Visualize all classifier plots
# For a complete documentation please see: https://docs.wandb.ai/guides/integrations/scikit
wandb.sklearn.plot_classifier(pipe.get_params()["classifier"],
                              full_pipeline_preprocessing.transform(x_train),
                              full_pipeline_preprocessing.transform(x_val),
                              y_train,
                              y_val,
                              predict,
                              pipe.predict_proba(x_val),
                              [0,1],
                              model_name='BestModel', feature_names=all_names)

wandb: 
wandb: Plotting BestModel.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


-----------------------------

### 4.6. Export the best model

In [ ]:
# types and names of the artifacts
artifact_type = "inference_artifact"
artifact_encoder = "target_encoder"
artifact_model = "model_export"

In [ ]:
logger.info("Dumping the artifacts to disk")
# Save the model using joblib
joblib.dump(pipe, artifact_model)

# Save the target encoder using joblib
joblib.dump(le, artifact_encoder)

29-05-2022 14:22:59 Dumping the artifacts to disk


['target_encoder']

In [ ]:
# Model artifact
artifact = wandb.Artifact(artifact_model,
                          type=artifact_type,
                          description="A full pipeline composed of a Preprocessing Stage and a Decision Tree model"
                          )

logger.info("Logging model artifact")
artifact.add_file(artifact_model)
run.log_artifact(artifact)

29-05-2022 14:23:00 Logging model artifact


In [ ]:
# Target encoder artifact
artifact = wandb.Artifact(artifact_encoder,
                          type=artifact_type,
                          description="The encoder used to encode the target variable"
                          )

logger.info("Logging target enconder artifact")
artifact.add_file(artifact_encoder)
run.log_artifact(artifact)

29-05-2022 14:23:00 Logging target enconder artifact


In [ ]:
run.finish()

Acc,0.80048
F1,0.52332
Precision,0.50998
Recall,0.53738


----------------------------

## 5.Testing

In [ ]:
run = wandb.init()

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    # Class Constructor
    def __init__(self, feature_names):
        self.feature_names = feature_names
        
    # Return self nothing to do here
    def fit(self, X, y=None):
        return self
    
    # Method that describes what tis custom transformer need to do
    def transform(self, X, y=None):
        return X[self.feature_names]
    
# Handling categorical features
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes one boolean as argument
    def __init__(self, new_features=True, colnames=None):
        self.new_features = new_features
        self.colnames = colnames
        
    # Return self nothing else to do here
    def fit(self, X, y=None):
        return self
    
    def get_feature_names_out(self):
        return self.colnames.tolist()
    
    # Transformer method we wrote for this transformer
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # Remove white space in categorical features
        df = df.apply(lambda row: row.str.strip())
        
        # customize feature?
        # How can I identify what needs to be modified? EDA!!!!!!
        if self.new_features:
            
            # replace ? with unknown
            edit_cols = ['Geography', 'Gender']
            for col in edit_cols:
                df.loc[df[col].str.contains("\?"), col] = 'unknown'
        
        # update column names
        self.colnames = df.columns
        df = pd.DataFrame(X, columns=self.colnames)
        
        return df

# transform numerical features
class NumericalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes a model parameter as its argument
    # model 0: minmax
    # model 1: standard
    # model 2: without scaler
    def __init__(self, model=0, colnames=None):
        self.model = model
        self.colnames = colnames
        self.scaler = None

    # Fit is used only to learn statistical about Scalers
    def fit(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # minmax
        if self.model == 0:
            self.scaler = MinMaxScaler()
            self.scaler.fit(df)
        # standard scaler
        elif self.model == 1:
            self.scaler = StandardScaler()
            self.scaler.fit(df)
        return self

    # return columns names after transformation
    def get_feature_names_out(self):
        return self.colnames

    # Transformer method we wrote for this transformer
    # Use fitted scalers
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # update columns name
        self.colnames = df.columns.tolist()

        # minmax
        if self.model == 0:
            # transform data
            df = self.scaler.transform(df)
        elif self.model == 1:
            # transform data
            df = self.scaler.transform(df)
        else:
            df = df.values

        return df

In [ ]:
# global variables

# name of the artifact related to test dataset
artifact_test_name = "churn_prediction_project/test.csv:latest"

# name of the model artifact
artifact_model_name = "churn_prediction_project/model_export:latest"

# name of the target encoder artifact
artifact_encoder_name = "churn_prediction_project/target_encoder:latest"

In [ ]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

In [ ]:
# initiate the wandb project
run = wandb.init(project="churn_prediction_project",job_type="test")

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


In [ ]:
logger.info("Downloading and reading test artifact")
test_data_path = run.use_artifact(artifact_test_name).file()
df_test = pd.read_csv(test_data_path)

# Extract the target from the features
logger.info("Extracting target from dataframe")
x_test = df_test.copy()
y_test = x_test.pop("Exited")

29-05-2022 14:23:32 Downloading and reading test artifact
29-05-2022 14:23:33 Extracting target from dataframe


In [ ]:
x_test.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,790,Spain,Male,37,6,0.00,2,1,1,119484.01
1,521,France,Male,35,6,96423.84,1,1,0,10488.44
2,712,France,Female,37,1,106881.50,2,0,0,169386.81
3,729,Spain,Female,38,10,0.00,2,1,0,189727.12
4,695,Germany,Male,52,8,103023.26,1,1,1,22485.64


In [ ]:
y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: Exited, dtype: int64

In [ ]:
# Extract the encoding of the target variable
logger.info("Extracting the encoding of the target variable")
encoder_export_path = run.use_artifact(artifact_encoder_name).file()
le = joblib.load(encoder_export_path)

29-05-2022 14:23:33 Extracting the encoding of the target variable


In [ ]:
y_test

0       0
1       0
2       0
3       0
4       0
       ..
2995    0
2996    0
2997    0
2998    1
2999    0
Name: Exited, Length: 3000, dtype: int64

In [ ]:
# Download inference artifact
logger.info("Downloading and load the exported model")
model_export_path = run.use_artifact(artifact_model_name).file()
pipe = joblib.load(model_export_path)

29-05-2022 14:23:33 Downloading and load the exported model


In [ ]:
# predict
logger.info("Infering")
predict = pipe.predict(x_test)

# Evaluation Metrics
logger.info("Test Evaluation metrics")
fbeta = fbeta_score(y_test, predict, beta=1, zero_division=1)
precision = precision_score(y_test, predict, zero_division=1)
recall = recall_score(y_test, predict, zero_division=1)
acc = accuracy_score(y_test, predict)

logger.info("Test Accuracy: {}".format(acc))
logger.info("Test Precision: {}".format(precision))
logger.info("Test Recall: {}".format(recall))
logger.info("Test F1: {}".format(fbeta))

run.summary["Acc"] = acc
run.summary["Precision"] = precision
run.summary["Recall"] = recall
run.summary["F1"] = fbeta

29-05-2022 14:23:34 Infering
29-05-2022 14:23:34 Test Evaluation metrics
29-05-2022 14:23:34 Test Accuracy: 0.7946666666666666
29-05-2022 14:23:34 Test Precision: 0.496206373292868
29-05-2022 14:23:34 Test Recall: 0.5351882160392799
29-05-2022 14:23:34 Test F1: 0.5149606299212598


In [ ]:
# Compare the accuracy, precision, recall with previous ones
print(classification_report(y_test,predict))

              precision    recall  f1-score   support

           0       0.88      0.86      0.87      2389
           1       0.50      0.54      0.51       611

    accuracy                           0.79      3000
   macro avg       0.69      0.70      0.69      3000
weighted avg       0.80      0.79      0.80      3000



In [ ]:
fig_confusion_matrix, ax = plt.subplots(1,1,figsize=(7,4))
ConfusionMatrixDisplay(confusion_matrix(predict,y_test,labels=[1,0]),
                       display_labels=["1","0"]).plot(values_format=".0f",ax=ax)

ax.set_xlabel("True Label")
ax.set_ylabel("Predicted Label")
plt.show()

In [ ]:
run.finish()

Acc,0.79467
F1,0.51496
Precision,0.49621
Recall,0.53519
